In [12]:
#!pip install pypdf

In [13]:
# 1. Imports
import os
import re
import pandas as pd
from pypdf import PdfReader
from typing import Dict, Any, Optional

In [14]:
# 2. CONFIGURAÇÃO E EXECUÇÃO

# Defina o nome da pasta onde os PDFs estão
PASTA_DOS_PDFS = "./artigos_para_extrair"
# Defina o nome do arquivo Excel que será gerado
ARQUIVO_DE_SAIDA = "relatorio_extracao_artigos.xlsx"

# Cria a pasta de PDFs se ela não existir
if not os.path.exists(PASTA_DOS_PDFS):
    print(f"Criando a pasta '{PASTA_DOS_PDFS}'. Por favor, coloque seus PDFs aqui.")
    os.makedirs(PASTA_DOS_PDFS)

else:
    print(f"Pasta de PDFs '{PASTA_DOS_PDFS}' pronta.")    

Pasta de PDFs './artigos_para_extrair' pronta.


**1. Função de Suporte**

In [15]:
# CÉLULA 2: FUNÇÕES DE SUPORTE (Lógica de Extração FINAL E DEFINITIVA)
def extrair_texto_das_primeiras_paginas(caminho_arquivo: str, num_paginas: int = 2) -> Optional[str]:
    """Extrai o texto das N primeiras páginas de um arquivo PDF."""
    texto_completo = ""
    try:
        reader = PdfReader(caminho_arquivo)
        limite = min(len(reader.pages), num_paginas)
        
        for i in range(limite):
            pagina = reader.pages[i]
            # Adiciona um marcador de quebra de página para auxiliar na extração contextual
            texto_completo += pagina.extract_text() + "\n--- QUEBRA_DE_PAGINA ---\n"
        
        return texto_completo
    except Exception as e:
        # print(f"Erro ao ler o PDF {caminho_arquivo}: {e}")
        return None

def extrair_dados_contextuais_generalizados(texto: str) -> Dict[str, Any]:
    """
    IMPLEMENTAÇÃO FINAL: Heurísticas robustas para delimitação de Título e limpeza de Autores.
    """
    dados = {
        "Título": "NÃO ENCONTRADO",
        "Autores": "NÃO ENCONTRADO",
        "Resumo": "NÃO ENCONTRADO",
        "e-mail": "NÃO ENCONTRADO",
        "Palavras-Chave": "NÃO ENCONTRADO"
    }
    
    flags = re.DOTALL | re.IGNORECASE
    
    # --- Passo 1: Extração de E-mail (Âncora de Posição) ---
    email_pattern = r"[\w\.-]+@[\w\.-]+(?:\.\w+)+"
    emails_encontrados = re.findall(email_pattern, texto)
    if emails_encontrados:
        dados["e-mail"] = ", ".join(emails_encontrados)

    # --- Passo 2 & 3: Resumo e Palavras-Chave (Extração e Delimitação) ---
    # Encontra o ponto de corte mais alto para o cabeçalho (e-mail ou Resumo)
    resumo_match = re.search(r"(Resumo|Abstract)\s*[:\n.]\s*(.+?)\s*(Palavras-chave|Keywords|\n\s*\d+\s*[A-ZÀ-Ú]+|\n\s*--- QUEBRA_DE_PAGINA ---)", texto, flags)
    if resumo_match:
        dados["Resumo"] = resumo_match.group(2).strip()
    
    palavras_chave_match = re.search(
        r"(Palavras-chave|Keywords)\s*[:\n.]\s*(.+?)\s*(?:\n\s*(Abstract|Resumo)\s*[:\n.]|\n\s*\d+\s*[A-ZÀ-Ú]+|\n\s*--- QUEBRA_DE_PAGINA ---)",
        texto,
        flags
    )
    if palavras_chave_match:
        dados["Palavras-Chave"] = palavras_chave_match.group(2).strip()


    # --- Passo 4: Extração de Título e Autores (Heurística Posicional REVISADA) ---
    
    # Define o ponto de corte do cabeçalho como o ponto mais alto encontrado (e-mail ou Resumo)
    indice_corte = len(texto) 
    if resumo_match:
        indice_corte = resumo_match.start()
    elif palavras_chave_match:
        indice_corte = palavras_chave_match.start()
    
    if emails_encontrados:
        indice_primeiro_email = texto.find(emails_encontrados[0])
        if 0 < indice_primeiro_email < indice_corte:
            indice_corte = indice_primeiro_email 

    texto_cabecalho = texto[:indice_corte].strip()
    
    # 4a. Heurística de Título: O título é a primeira grande porção de texto em caixa alta/versaletes
    # Procura um bloco de texto que termine antes de algo que se pareça com o nome de um autor
    titulo_e_autores_match = re.search(
        r"(.+?)\n\s*([A-ZÀ-Ú][a-zà-ú]+ [A-ZÀ-Ú][a-zà-ú]+[^\n]*)$", 
        texto_cabecalho, 
        flags=re.DOTALL | re.MULTILINE
    )
    
    if titulo_e_autores_match:
        # TÍTULO: O primeiro grupo de captura (o que vem antes do primeiro nome)
        titulo_bloco = titulo_e_autores_match.group(1).strip()
        titulo_linhas = [
            linha.strip() for linha in titulo_bloco.split('\n') 
            if linha.strip() and not any(termo in linha for termo in ['ISSN', 'Revista', 'v.', 'n.', 'DOI', 'Maio', 'CC', 'SA', '202', 'DISTANCE EDUCATION'])
        ]
        if titulo_linhas:
            dados["Título"] = " ".join(titulo_linhas)

        # 4b. AUTORES: O bloco de autores/afiliações é o que resta no cabeçalho após o Título.
        autores_bloco_raw = texto_cabecalho[titulo_e_autores_match.end(1):].strip()
        
        if autores_bloco_raw:
            # CORREÇÃO CRÍTICA: Limpeza de Autores (lida com hífen, en-dash, e afiliações)
            
            # Remove afiliações tipo ' – UNINTER', ' - UNINTER', ' -UFBA' e parênteses
            # O padrão agora lida com o hífen/en-dash seguido por espaços e caixa alta
            autores_limpos = re.sub(r'\s*[\-–]\s*[A-ZÀ-Ú\s]+|\([^\)]*\)|\s*Doutor.*|\s*Professor.*|\s*PhD.*', '', autores_bloco_raw, flags=flags).strip()
            
            # Divide o bloco por ; (separador mais comum entre autores/afiliações) ou \n
            if ';' in autores_limpos:
                partes = autores_limpos.split(';')
            else:
                partes = autores_limpos.split('\n')

            nomes = []
            for parte in partes:
                parte = parte.strip()
                # Verifica se o texto restante ainda se parece com um nome (pelo menos 2 palavras, começa com Maiúscula)
                if re.search(r'^[A-ZÀ-Ú][a-zà-ú]+ [A-ZÀ-Ú]', parte) and len(parte.split()) >= 2:
                    nomes.append(parte)

            if nomes:
                dados["Autores"] = ", ".join(nomes)

    return dados

In [16]:
# texto_teste = extrai_texto_das_primeiras_paginas("./artigos_para_extrair/5530_Versão_final_do_artigo_EDUCAÇÃO_À_DISTÂNCIA_NA_FORMAÇÃO_EM_SERVIÇO_SOCIAL_ UMA_ESTRATÉGIA_DE_DEMOCRATIZAÇÃO_DE_ACESSOS.pdf")

In [17]:
# 5. Função de Automação Principal

def automatizar_extracao(pasta_pdfs: str, nome_arquivo_saida: str):
    """Intera por todos os PDFs, extrai os dados e gera o arquivo XLSX."""

    lista_resultados = []
    arquivos_pdf = [f for f in os.listdir(pasta_pdfs) if f.endswith(".pdf")]

    if not arquivos_pdf:
        print(f"Nenhum arquivo .pdf encontrado na pasta: {pasta_pdfs}. Coloque os arquivos na pasta {pasta_pdfs} e tente novamente.")
        return

    # Processamento em lote (loop)
    print(f"Iniciando o processamento de {len(arquivos_pdf)} artigos...")
    for nome_arquivo in arquivos_pdf:
        caminho_completo = os.path.join(pasta_pdfs, nome_arquivo)
        print(f" ->Processando: {nome_arquivo}...")

        # Etapa 1: Extrair texto do PDF
        texto = extrair_texto_das_primeiras_paginas(caminho_completo, num_paginas=4)

        dados = {"Nome do Arquivo": nome_arquivo, "Status": "Sucesso"}

        if texto:
            # Etapa 2: Aplicar o algorítmo de Extração
            dados.update(extrair_dados_contextuais_generalizados(texto))
        else:
            dados.update({
                "Título": "NÃO ENCONTRADO", "Autores": "NÃO ENCONTRADO",
                "Resumo": "NÃO ENCONTRADO", "e-mail": "NÃO ENCONTRADO",
                "Palavras-Chave": "NÃO ENCONTRADO",
                "Status": "Erro na leitura do PDF"
            })

        lista_resultados.append(dados)

    # Etapa 3: Estruturar e gerar XLSX
    if lista_resultados:
        df = pd.DataFrame(lista_resultados)

        # COLUNAS ORDENADAS E ATUALIZADAS
        colunas_ordenadas = ['Nome do Arquivo', 'Status', 'Título', 'Autores', 'Resumo', 'Palavras-Chave', 'e-mail']

        # Garante a ordem e exporta
        df = df.reindex(columns=colunas_ordenadas, fill_value='NÃO ENCONTRADO')
        df.to_excel(nome_arquivo_saida, index=False)
        
        print("\n" + "="*70)
        print(f"Processo Concluído! {len(arquivos_pdf)} artigos processados. Dados salvos em: {nome_arquivo_saida}")
        print("Amostra dos resultados (DataFrame):")
        #Exibe a tabela diretamente no notebook
        display(df)
        print("\n" + "="*70)        

In [18]:
automatizar_extracao(PASTA_DOS_PDFS, ARQUIVO_DE_SAIDA)

Iniciando o processamento de 9 artigos...
 ->Processando: 5505.pdf...
 ->Processando: 5530.pdf...
 ->Processando: 5723.pdf...
 ->Processando: 5977.pdf...
 ->Processando: 5985.pdf...
 ->Processando: 6041.pdf...
 ->Processando: 6100.pdf...
 ->Processando: 6127.pdf...
 ->Processando: 6208.pdf...

Processo Concluído! 9 artigos processados. Dados salvos em: relatorio_extracao_artigos.xlsx
Amostra dos resultados (DataFrame):


,Nome do Arquivo,Status,Título,Autores,Resumo,Palavras-Chave,e-mail
0,5505.pdf,Sucesso,ACESSO E A PERMANÊNCIA NA EDUCAÇÃO A DISTÂNCIA...,"Gisele Cordeiro do Rocio, Janice Mendes da Silva<","O presente estudo analisa o acesso, permanênci...","Educação a distância, qualidade, acesso, perma...","gisele.c@uninter.com, janice.s@uninter.com"
1,5530.pdf,Sucesso,EDUCAÇÃO À DISTÂNCIA NA FORMAÇÃO EM SERVIÇO SO...,"Neiva Silvana Hack, Adriane Bührer Baglioli Br...",O presente trabalho propõe a discussão da impo...,Democratização; Formação profissional; Interio...,"neiva.h@uninter.com, adriane.b@uninter.com, cl..."
2,5723.pdf,Sucesso,INTERNACIONALIZAÇÃO DAS IES NA MODALIDADE EaD ...,Angela Cristina Kochinski Tripoli.,Este artigo explora a internacionalização das ...,Educação a Distância (EaD); Internacionalizaçã...,"angela.t@uninter.com, virginia.c@uninter.com, ..."
3,5977.pdf,Sucesso,NÃO ENCONTRADO,NÃO ENCONTRADO,Este ensaio aborda conceitos da Educação ...,Distance education; online education; educ...,"carla.alexandre@ufpe.br, patricia.cavalcante@u..."
4,5985.pdf,Sucesso,PRESENCIALIDADES OUTRAS NA EDUCAÇÃO A DISTÂNCI...,"Luiz Carlos Sacramento da Luz, Maria Thereza Á...",A Educação a Distância (EaD) no Brasil está em...,educação a distância; presencialidade; metapre...,"dnluiz@gmail.com, maria.thereza@ufba.br, eniel..."
5,6041.pdf,Sucesso,Programa de Aprendizagem na modalidade EAD van...,Jéssica Fonseca de Oliveira<,O foco do presente estudo foi a Lei 10.097 de ...,Lei da aprendizagem; Educação a distância; amp...,jessicafo@cieerj.org.br
6,6100.pdf,Sucesso,NÃO ENCONTRADO,NÃO ENCONTRADO,NÃO ENCONTRADO,NÃO ENCONTRADO,NÃO ENCONTRADO
7,6127.pdf,Sucesso,"INTERNACIONALIZAÇÃO PEDAGÓGICA, EDUCAÇÃO A D...","Adriana Aparecida de Lima Terçariol, São Paulo...",O artigo apresenta um recorte de uma formação ...,Formação inicial e continuada; Educação STEAM;...,"atercariol@gmail.com, stephani.vilela@gmail.co..."
8,6208.pdf,Sucesso,1 TRANSDISCIPLINARIDADE: AVANÇOS NECESSÁRIOS P...,Margarete Terezinha de Andrade Costa,Este artigo apresenta a abordagem transdiscipl...,transdisciplinaridade; educação a distância; p...,"margarete.c@uninter.com, rodrigo.s@uninter.com"


In [19]:
PASTA_DOS_PDFS

'./artigos_para_extrair'

In [20]:
ARQUIVO_DE_SAIDA

'relatorio_extracao_artigos.xlsx'